最大回撤计算

In [5]:
!pip install pyecharts -i http://pypi.douban.com/simple/ --trusted-host pypi.douban.com

Looking in indexes: http://pypi.douban.com/simple/


In [6]:
!pip list

Package                            Version
---------------------------------- -----------
alabaster                          0.7.12
anaconda-client                    1.7.2
anaconda-navigator                 2.0.3
anaconda-project                   0.9.1
anyio                              2.2.0
appdirs                            1.4.4
argh                               0.26.2
argon2-cffi                        20.1.0
asn1crypto                         1.4.0
astroid                            2.5
astropy                            4.2.1
async-generator                    1.10
atomicwrites                       1.4.0
attrs                              20.3.0
autopep8                           1.5.6
Babel                              2.9.0
backcall                           0.2.0
backports.functools-lru-cache      1.6.4
backports.os                       0.1.1
backports.shutil-get-terminal-size 1.0.0
backports.tempfile                 1.0
backports.weakref                  1.0.post1
baidu

In [33]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import openpyxl
import time
import empyrical
#import tushare as ts


In [34]:
def max_drawdown(df):
    
    #temp = df.index
    xdate = df.index
    ycapital = df.iloc[:, 0]
    # 计算每天的回撤
    drawdown = [0]
    tmp_max_capital = ycapital[0]
    for c in df.iloc[:, 0]:
        tmp_max_capital = max(c, tmp_max_capital)
        drawdown.append(1 - c / tmp_max_capital)
    # 最大回撤
    MaxDrawdown = max(drawdown)
    # 计算最大回撤日期范围
    endidx = np.argmax(drawdown)
    if not endidx:
        return "NA", "NA", "NA", "NA", "NA"
    startidx = np.argmax(ycapital[:endidx])
    start_date = xdate[startidx]
    end_date = xdate[endidx - 1]
    restore_endidx = np.inf
    for t in range(endidx, len(xdate)):
        if ycapital[t] >= ycapital[startidx]:
            restore_endidx = t
            break
        
    restore_endidx = min(restore_endidx, len(xdate) - 1)

    if xdate[restore_endidx]==xdate[-1]:
        restore_end = "NA"
        restore_duration = "NA"
    else:
        restore_end = xdate[restore_endidx].strftime('%Y-%m-%d')
        restore_duration = (xdate[restore_endidx] - xdate[endidx - 1]).days
     
    return MaxDrawdown, start_date.date(), (end_date-start_date).days, end_date.date(), restore_duration

In [36]:
# to drawdown data

if not os.path.exists(r'D:\test-20220815\anxin_hjm\最大回撤数据汇总'):
    os.mkdir(r'D:\test-20220815\anxin_hjm\最大回撤数据汇总')

for sheet in pd.read_excel(r'D:\test-20220815\anxin_hjm\最大回撤数据汇总\0610.xlsx', None).keys():
    to_sheet = pd.DataFrame(columns=["产品", "时间", "最大回撤", "回撤开始时间", "回撤持续时长", "最大回撤发生时间", "最大回撤回补周期"])
    df = pd.read_excel(r'D:\test-20220815\anxin_hjm\最大回撤数据汇总\0610.xlsx',sheet_name = sheet)
    names = np.delete(df.columns.values,0)

    for name in names:
        df_prod = df[['日期',name]].set_index("日期").dropna()
        print(df_prod)
        try:
            maxdrawdown, start_date, drawdown_duration, drawdown_date, retrace_duration= max_drawdown(df_prod)
            to_sheet = to_sheet.append(
                {"产品": name, "时间": "全周期", "最大回撤": maxdrawdown, "回撤开始时间": start_date, "回撤持续时长": drawdown_duration,
                 "最大回撤发生时间": drawdown_date, "最大回撤回补周期": retrace_duration}, ignore_index=True)
            # to_sheet_full = to_sheet_full.append(
            #     {"产品": name, "最大回撤": maxdrawdown, "回撤开始时间": start_date, "最大回撤发生时间": drawdown_date}, ignore_index=True)

            for year in (df_prod.index.year).unique():
                df_prod_year = df_prod[df_prod.index.year == year]
                maxdrawdown, start_date, drawdown_duration, drawdown_date, retrace_duration = max_drawdown(df_prod_year)
                to_sheet = to_sheet.append(
                    {"产品": name, "时间": year, "最大回撤": maxdrawdown, "回撤开始时间": start_date, "回撤持续时长": drawdown_duration,
                     "最大回撤发生时间": drawdown_date, "最大回撤回补周期": retrace_duration}, ignore_index=True)

            to_sheet.to_excel(f'D:/test-20220815/anxin_hjm/最大回撤数据汇总/{sheet}.xlsx', index=False)
        except:
            pass


            超量子沪深300增强1号
日期                      
2020-07-17      1.000000
2020-07-24      1.008479
2020-07-31      0.970398
2020-08-07      0.972570
2020-08-14      0.972382
...                  ...
2022-05-06      1.059666
2022-05-13      1.074134
2022-05-20      1.077215
2022-05-27      1.086740
2022-06-02      1.097400

[98 rows x 1 columns]
                大道丰盛
日期                  
2021-12-03  1.000000
2021-12-10  0.982518
2021-12-17  0.990143
2021-12-24  0.995471
2021-12-31  0.997395
2022-01-07  1.000544
2022-01-14  1.000034
2022-01-21  0.997447
2022-01-28  0.998448
2022-02-11  1.009579
2022-02-18  1.007829
2022-02-25  1.014681
2022-03-04  1.019402
2022-03-11  1.021416
2022-03-18  1.013550
2022-03-25  1.020097
2022-04-01  1.022360
2022-04-08  1.024402
2022-04-15  1.024491
2022-04-22  1.024559
2022-04-29  1.026727
2022-05-06  1.036499
2022-05-13  1.039499
2022-05-20  1.036538
2022-05-27  1.042084
2022-06-02  1.039502
            黑翼风行三号沪深300指数增强
日期                         
2018-01-

最大回撤-Visualization

In [44]:
import pandas as pd
import numpy as np
import pyecharts.options as opts
import random
import datetime
import pyecharts.options as opts
from pyecharts.charts import Line,Grid,Page,Bar,Tab
from pyecharts.commons.utils import JsCode
from pyecharts.components import Table
from pyecharts.faker import Faker
import pyecharts as pyc
from pyecharts import options as opts
from pyecharts.charts import Bar, Timeline
import os
import re

In [30]:
def plot(df, start, end, prod_line):
    
    for idx in range(df.shape[0]):
        row = df.iloc[idx,]
        
        prod_line.set_global_opts(title_opts=opts.TitleOpts(title=strategy[:-5]+str(period)+"MDD-%"), legend_opts = opts.LegendOpts(is_show=True, type_="scroll", pos_left='25%'), 
        xaxis_opts=opts.AxisOpts(type_="time", name_gap=45, name_rotate=350, min_=start.date(), max_=end.date()), yaxis_opts=opts.AxisOpts(max_=df["最大回撤"]*100+1), 
        tooltip_opts=opts.TooltipOpts(trigger="axis"))
    
        prod_line.add_xaxis([i.date() for i in pd.date_range(row["回撤开始时间"], row["最大回撤发生时间"])])
        prod_line.add_yaxis(row["产品"], [row["最大回撤"]*100]*((row["最大回撤发生时间"]-row["回撤开始时间"]).days+1), linestyle_opts=opts.LineStyleOpts(width=1)).set_series_opts(label_opts=opts.LabelOpts(is_show=False))
  
    return prod_line


In [45]:
# plot all
periods = ["全周期", 2022, 2021, 2020, 2019, 2018]

for strategy in os.listdir(r'D:/test-20220815/anxin_hjm/最大回撤数据汇总'):
    print(strategy)
    strategy_df = pd.read_excel(f'D:/test-20220815/anxin_hjm/最大回撤数据汇总/{strategy}')
    page = Page(page_title=strategy[:-5]+"MDD") #each html
    for period in periods:
        df_period = strategy_df[strategy_df["时间"]==period]
        df_period = df_period[["产品", "最大回撤", "回撤开始时间", "最大回撤发生时间"]]
        df_period = df_period.dropna()
        if period=="全周期":
            start = datetime.datetime(2018, 1, 1)
            end = datetime.datetime(2022, 6, 22)
        elif period==2022:
            start = datetime.datetime(2022, 1, 1)
            end = datetime.datetime(2022, 6, 22)
        else:
            start = datetime.datetime(period, 1, 1)
            end = datetime.datetime(period, 12, 31)

        timeline=Timeline(init_opts=opts.InitOpts(width="1700px", height="750px", page_title=period)) #each period/diagram
        for prod_idx in range(df_period.shape[0]):
            prod = df_period.iloc[prod_idx,]
            line=Line(init_opts=opts.InitOpts(width="1700px", height="750px",page_title="MDD"))
            line.add_xaxis([i.date() for i in pd.date_range(prod["回撤开始时间"], prod["最大回撤发生时间"])])
            line.add_yaxis(prod["产品"], [prod["最大回撤"]*100]*((prod["最大回撤发生时间"]-prod["回撤开始时间"]).days+1),
            linestyle_opts=opts.LineStyleOpts(width=10), itemstyle_opts=opts.ItemStyleOpts(border_width=3))
            line.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
            line = plot(df_period, start, end, line)
            timeline.add(line, prod["产品"])
        page.add(timeline)

    page.render(f"{strategy[:-5]}.html")


1000超额.xlsx
300超额.xlsx
500超额.xlsx
中性中短周期.xlsx
中性中长周期.xlsx
量化多头（净值）.xlsx


In [40]:
#hereeeee
# plot cb-500

# plot cb MDD in red
def plot_cb(df_cb, line, line_color):
    for cb_idx in range(df_cb.shape[0]):
        row_cb = df_cb.iloc[cb_idx,]
        line.add_xaxis([i.date() for i in pd.date_range(row_cb["回撤开始时间"], row_cb["最大回撤发生时间"])])
        line.add_yaxis(row_cb["产品"], [row_cb["最大回撤"]*100]*((row_cb["最大回撤发生时间"]-row_cb["回撤开始时间"]).days+1), color=line_color, linestyle_opts=opts.LineStyleOpts(width=3), itemstyle_opts=opts.ItemStyleOpts(border_width=2))
        line.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    return line



periods = ["全周期", 2022, 2021, 2020, 2019, 2018]

# df_cb = pd.read_excel(r'D:\test-20220815\anxin_hjm\最大回撤数据汇总\可转债.xlsx')
df_500 = pd.read_excel(r'D:\test-20220815\anxin_hjm\最大回撤数据汇总\500超额.xlsx')

page = Page("可转债-500超额MDD") #each html

for period in periods:

    period_cb = df_cb[df_cb["时间"]==period]
    period_500 = df_500[df_500["时间"]==period]

    period_cb = period_cb[["产品", "最大回撤", "回撤开始时间", "最大回撤发生时间"]]
    period_500 = period_500[["产品", "最大回撤", "回撤开始时间", "最大回撤发生时间"]]
        

    period_cb = period_cb.dropna()
    period_500 = period_500.dropna()


    if period=="全周期":
        start = datetime.datetime(2018, 1, 1)
        end = datetime.datetime(2022, 6, 22)
    elif period==2022:
        start = datetime.datetime(2022, 1, 1)
        end = datetime.datetime(2022, 6, 22)
    else:
        start = datetime.datetime(period, 1, 1)
        end = datetime.datetime(period, 12, 31)

    

    timeline=Timeline(init_opts=opts.InitOpts(width="1700px", height="750px", page_title="可转债")) #each period/diagram
    line=Line(init_opts=opts.InitOpts(width="1700px", height="750px",page_title="MDD"))
    for prod_idx in range(period_cb.shape[0]):
        prod = period_cb.iloc[prod_idx,]
        line.add_xaxis([i.date() for i in pd.date_range(prod["回撤开始时间"], prod["最大回撤发生时间"])])
        line.add_yaxis(prod["产品"], [prod["最大回撤"]*100]*((prod["最大回撤发生时间"]-prod["回撤开始时间"]).days+1), 
        linestyle_opts=opts.LineStyleOpts(width=10), itemstyle_opts=opts.ItemStyleOpts(border_width=6), color="red")
        
    line.set_global_opts(title_opts=opts.TitleOpts(title=str(period)+"MDD-%"), legend_opts = opts.LegendOpts(is_show=True, type_="scroll", pos_left='25%'), 
    xaxis_opts=opts.AxisOpts(type_="time", min_=start.date(), max_=end.date()), yaxis_opts=opts.AxisOpts(max_=round(max(df_cb["最大回撤"].max(), df_500["最大回撤"].max())*100+3)), 
    tooltip_opts=opts.TooltipOpts(trigger="axis"))
        
    line = plot_cb(period_500, line, "#6e9ef1")
            
    timeline.add(line, "可转债")
    line=Line(init_opts=opts.InitOpts(width="1700px", height="750px",page_title="MDD"))
    
    for prod_idx in range(period_500.shape[0]):
        prod = period_500.iloc[prod_idx,]
        line.add_xaxis([i.date() for i in pd.date_range(prod["回撤开始时间"], prod["最大回撤发生时间"])])
        line.add_yaxis(prod["产品"], [prod["最大回撤"]*100]*((prod["最大回撤发生时间"]-prod["回撤开始时间"]).days+1), 
        linestyle_opts=opts.LineStyleOpts(width=10), itemstyle_opts=opts.ItemStyleOpts(border_width=8), color="blue")
    
    line = plot_cb(period_cb, line, "red")
    timeline.add(line, "500超额")
    


    page.add(timeline)
page.render("可转债-500超额MDD.html")
    

NameError: name 'df_cb' is not defined

In [ ]:

# plot cb-500
periods = ["全周期", 2022, 2021, 2020, 2019, 2018]

df_cb = pd.read_excel(r'D:\test\anxin_hjm\最大回撤数据汇总\可转债.xlsx')
df_500 = pd.read_excel(r'D:\test\anxin_hjm\最大回撤数据汇总\500超额.xlsx')

page = Page("可转债-500超额MDD") #each html

for period in periods:

    period_cb = df_cb[df_cb["时间"]==period]
    period_500 = df_500[df_500["时间"]==period]

    period_cb = period_cb[["产品", "最大回撤", "回撤开始时间", "最大回撤发生时间"]]
    period_500 = period_500[["产品", "最大回撤", "回撤开始时间", "最大回撤发生时间"]]
        

    period_cb = period_cb.dropna()
    period_500 = period_500.dropna()

    timeline=Timeline(init_opts=opts.InitOpts(width="1700px", height="750px", page_title=period)) #each period/diagram
    if period=="全周期":
        start = datetime.datetime(2018, 1, 1)
        end = datetime.datetime(2022, 6, 22)
    elif period==2022:
        start = datetime.datetime(2022, 1, 1)
        end = datetime.datetime(2022, 6, 22)
    else:
        start = datetime.datetime(period, 1, 1)
        end = datetime.datetime(period, 12, 31)

    #timeline=Timeline(init_opts=opts.InitOpts(width="1700px", height="750px", page_title=period)) #each period/diagram
    line=Line(init_opts=opts.InitOpts(width="1700px", height="750px",page_title="MDD"))
    
    line.set_global_opts(title_opts=opts.TitleOpts(title=str(period)+"MDD-%"), legend_opts = opts.LegendOpts(is_show=True, type_="scroll", pos_left='25%'), 
        xaxis_opts=opts.AxisOpts(type_="time", min_=start.date(), max_=end.date()), yaxis_opts=opts.AxisOpts(max_=round(max(df_cb["最大回撤"].max(), df_500["最大回撤"].max())*100+3)), 
        tooltip_opts=opts.TooltipOpts(trigger="axis"))

    line = plot_cb(period_500, line, "#6e9ef1")
    line = plot_cb(period_cb, line, "red")
    


    page.add(line)
page.render("可转债-500超额MDD.html")
    

In [ ]:


# 月胜率
def monthly_rate(value):
    monthly_series = empyrical.aggregate_returns(value[i], 'monthly')
    monthly_df = monthly_series.to_frame()
    monthly_df['positive'] = monthly_df[str(i)] > 0
    win_month = (monthly_df.positive).sum()
    lose_month = (~monthly_df.positive).sum()
    rate = win_month / (win_month + lose_month) * 100
    return rate

# 周胜率
def weekly_rate(value):
    win_rate = value.loc[value[str(i)] > 0].shape[0]
    rate = win_rate / value.shape[0] * 100
    return rate

# 夏普比率
def sharpe_ratio(df):
    anl_std = df.std() * (52 ** 0.5)
    sharpe = aggregate_return_yearly[j] / anl_std
    return sharpe

# 全周期夏普比率
def full_sharpe_ratio(df):
    anl_std = df.std() * (52 ** 0.5)
    sharpe = full_return_yearly.iloc[0] / anl_std
    return sharpe

# 索缇诺比率
def sortino_ratio(df):
    temp = df.loc[df[str(i)] < 0] ** 2
    temp = temp.sum()
    temp = (temp / (df.dropna().shape[0] - 1)) ** 0.5
    temp = temp * (52 ** 0.5)
    sortino = aggregate_return_yearly[j] / temp
    return sortino

# 全周期索缇诺比率
def full_sortino_ratio(df):
    temp = df.loc[df[str(i)] < 0] ** 2
    temp = temp.sum()
    temp = (temp / (df.dropna().shape[0] - 1)) ** 0.5
    #print('n',(value[str(i)].shape[0] - 1))
    temp = temp * (52 ** 0.5)
    sortino = full_return_yearly.iloc[0] / temp
    return sortino


# 索缇诺星比率
def sortino_ratio_star(df):
    temp = df.loc[df[str(i)] < 0] ** 2
    temp_new = df.loc[df[str(i)] < 0]
    temp = temp.sum()
    temp = (temp / (temp_new.shape[0] - 1)) ** 0.5
    temp = temp * (52 ** 0.5)
    sortino_star = (full_return_yearly[j]) / temp
    return sortino_star


# 全周期索缇诺星比率
def sortino_ratio_star(df):
    temp = df.loc[df[str(i)] < 0] ** 2
    temp_new = df.loc[df[str(i)] < 0]
    temp = temp.sum()
    temp = (temp / (temp_new.shape[0] - 1)) ** 0.5
    temp = temp * (52 ** 0.5)
    sortino_star = (full_return_yearly.iloc[0]) / temp
    return sortino_star


# 下行标准差
def s_std(df):
    temp = df.loc[df[i] < 0] ** 2
    temp = temp.sum()
    temp = (temp / (df.dropna().shape[0] - 1)) ** 0.5
    temp = temp * (52 ** 0.5)
    return temp

# 下行标准差星比率
def s_std_star(df):
    temp = df.loc[df[i] < 0] ** 2
    temp_new = df.loc[df[i] < 0]
    temp = temp.sum()
    temp = (temp / (temp_new.shape[0] - 1)) ** 0.5
    temp = temp * (52 ** 0.5)
    return temp



def to_per(num, place):
    if place==0:
        return str(round(num*100))+"%"
    else:
        return str(round(num*100, place))+"%"



def insert_median(df):
    for col in df.columns:
        col_median = df[col]


    

In [ ]:

# to value indexes

for k in pd.read_excel(r'D:\test\anxin_hjm\最大回撤数据汇总\0610.xlsx', None).keys():
    write = pd.ExcelWriter('D:\test\anxin_hjm\最大回撤数据汇总\净值月报\"+k+".xlsx')


    df = pd.read_excel('../data/0610.xlsx',sheet_name = k)
    name = np.delete(df.columns.values,0)
    weekly_return = pd.DataFrame()
    monthly_return = pd.DataFrame()
    result = pd.DataFrame()

    for i in name:
        df1 = df.loc[:, ['日期',i]]
        df1 = df1.dropna()
        df1 = df1.reset_index(drop=True)
        df1 = df1.set_index('日期')
        df2 = df1.copy()

        df1['exe_return'] = df1[i]
        df1['prev_return'] = df1[i].shift(1)
        df1[i] = (df1['exe_return'] - df1['prev_return']) / df1['prev_return']
        df1 = df1.drop(['prev_return', 'exe_return'], axis=1)
        df1 = df1.dropna()

        new_weekly_return = empyrical.aggregate_returns(df1[i], 'weekly')
        weekly_return = weekly_return.append(new_weekly_return)

        new_monthly_return = empyrical.aggregate_returns(df1[i], 'monthly')
        monthly_return = monthly_return.append(new_monthly_return)

        for j in (df1.index.year).unique():
            aggregate_return_yearly = empyrical.aggregate_returns(df1.loc[str(j)][str(i)], 'yearly')/ df1.loc[str(j)].shape[0] * 52
            aggregate_return = empyrical.aggregate_returns(df1.loc[str(j)][str(i)], 'yearly')

            # 年化波动率
            anl_std = df1[str(i)][str(j)].std() * (52 ** 0.5)
            df_year = df1.loc[str(j)]


            std = s_std(df_year)
            std_star = s_std_star(df_year)


            monthly = monthly_rate(df_year)
            weekly = weekly_rate(df_year)

            sharpe = sharpe_ratio(df_year)
            sortino = sortino_ratio(df_year)
            sortino_star = sortino_ratio(df_year)
        
            max_dd = empyrical.max_drawdown(df1[str(i)][str(j)])
            calmar = -1 * (aggregate_return_yearly / max_dd)
            
            df_mdd = df2[df2.index.year==j]
            MaxDrawdown, start_date, end_date, restore_end, restore_duration = max_drawdown(df_mdd)

            if MaxDrawdown!= "NA":
                MaxDrawdown = str(round(MaxDrawdown*100,2))+"%"
            
            result = result.append({'产品': i, '年份':j, '收益率': to_per(aggregate_return.iloc[0], 2), '年化收益率': to_per(aggregate_return_yearly.iloc[0], 2),  '年化波动率': to_per(anl_std, 2), 
             '年化下行标准差': to_per(std[0], 2), '年化下行标准差*': to_per(std_star[0], 2), "最大回撤": MaxDrawdown, "回补周期(天)": restore_duration, 
             '夏普比率': round(sharpe.iloc[0], 2), '索提诺': round(sortino.iloc[0], 2), '索提诺*': round(sortino_star.iloc[0], 2), '卡玛比率':round(calmar.iloc[0], 2), 
             '月胜率': str(round(monthly))+"%",'周胜率': str(round(weekly))+"%"}, ignore_index=True)


        full_return_yearly = empyrical.aggregate_returns(df1[i], 'yearly')/ df1.shape[0] * 52
        full_return = empyrical.aggregate_returns(df1[i], 'yearly')
        full_anl_std = df1[i].std() * (52 ** 0.5)
        full_std = s_std(df1)
        full_std_star = s_std_star(df1)
        full_MaxDrawdown, start_date, end_date, restore_end, full_restore_duration = max_drawdown(df2)
        
        full_sharpe = full_sharpe_ratio(df1)
        full_sortino = full_sortino_ratio(df1)
        full_sortino_star = full_sortino_ratio(df1)

        
        full_max_dd = empyrical.max_drawdown(df1[str(i)])
        full_calmar = -1 * (full_return_yearly / full_max_dd)


        full_monthly = monthly_rate(df1)
        full_weekly = weekly_rate(df1)

        if full_MaxDrawdown!= "NA":
                full_MaxDrawdown = str(round(full_MaxDrawdown*100,2))+"%"


        result = result.append({'产品': i, '年份':"全周期", '收益率': to_per(full_return.iloc[0], 2), '年化收益率': to_per(full_return_yearly.iloc[0], 2),  '年化波动率': to_per(full_anl_std, 2), 
        '年化下行标准差': to_per(full_std[0], 2), '年化下行标准差*': to_per(full_std_star[0], 2),  "最大回撤": full_MaxDrawdown, "回补周期(天)": full_restore_duration, 
        '夏普比率': round(sharpe.iloc[0], 2), '索提诺': round(full_sortino.iloc[0], 2), '索提诺*': round(full_sortino_star.iloc[0], 2), '卡玛比率':round(full_calmar.iloc[0], 2), 
        '月胜率': str(round(monthly))+"%", '周胜率': str(round(weekly))+"%"}, ignore_index=True)
    
    
    result_levels = result.set_index(['年份', '产品'])

    result_levels = result_levels.unstack(0)
    print(result_levels.T.swaplevel(0, 1).sort_index(axis=0, level='年份', ascending=False, sort_remaining=False))
    #print(result_levels.T.swaplevel(0, 1).sort_index(level='年份', ascending=False, sort_remaining=False))
    result_levels = result_levels.T.swaplevel(0, 1).sort_index(level='年份', ascending=False, sort_remaining=False).T
    result_levels.to_excel(write, sheet_name="指标", index=True) 
    weekly_return.sort_index(axis=1, ascending=False).to_excel(write, sheet_name="weekly return")
    monthly_return.sort_index(axis=1,  ascending=False).to_excel(write, sheet_name="monthly return")
    write.save()
    break
    

In [ ]:

# to value indexes

from pandas.api.types import CategoricalDtype
for k in pd.read_excel(r'D:\test\anxin_hjm\最大回撤数据汇总\0610.xlsx', None).keys():
    write = pd.ExcelWriter("..\\data\\净值月报\\"+k+".xlsx")


    df = pd.read_excel('../data/0610.xlsx',sheet_name = k)
    name = np.delete(df.columns.values,0)
    weekly_return = pd.DataFrame()
    monthly_return = pd.DataFrame()
    result = pd.DataFrame()

    for i in name:
        df1 = df.loc[:, ['日期',i]]
        df1 = df1.dropna()
        df1 = df1.reset_index(drop=True)
        df1 = df1.set_index('日期')
        df2 = df1.copy()

        df1['exe_return'] = df1[i]
        df1['prev_return'] = df1[i].shift(1)
        df1[i] = (df1['exe_return'] - df1['prev_return']) / df1['prev_return']
        df1 = df1.drop(['prev_return', 'exe_return'], axis=1)
        df1 = df1.dropna()

        new_weekly_return = empyrical.aggregate_returns(df1[i], 'weekly')
        weekly_return = weekly_return.append(new_weekly_return)

        new_monthly_return = empyrical.aggregate_returns(df1[i], 'monthly')
        monthly_return = monthly_return.append(new_monthly_return)

        for j in (df1.index.year).unique():
            aggregate_return_yearly = empyrical.aggregate_returns(df1.loc[str(j)][str(i)], 'yearly')/ df1.loc[str(j)].shape[0] * 52
            aggregate_return = empyrical.aggregate_returns(df1.loc[str(j)][str(i)], 'yearly')

            # 年化波动率
            anl_std = df1[str(i)][str(j)].std() * (52 ** 0.5)
            df_year = df1.loc[str(j)]


            std = s_std(df_year)
            std_star = s_std_star(df_year)


            monthly = monthly_rate(df_year)
            weekly = weekly_rate(df_year)

            sharpe = sharpe_ratio(df_year)
            sortino = sortino_ratio(df_year)
            sortino_star = sortino_ratio(df_year)
        
            max_dd = empyrical.max_drawdown(df1[str(i)][str(j)])
            calmar = -1 * (aggregate_return_yearly / max_dd)
            
            df_mdd = df2[df2.index.year==j]
            MaxDrawdown, start_date, end_date, restore_end, restore_duration = max_drawdown(df_mdd)

            if MaxDrawdown!= "NA":
                MaxDrawdown = str(round(MaxDrawdown*100,2))+"%"
            
            result = result.append({'产品': i, '年份':j, '收益率': to_per(aggregate_return.iloc[0], 2), '年化收益率': to_per(aggregate_return_yearly.iloc[0], 2),  '年化波动率': to_per(anl_std, 2), 
             '年化下行标准差': to_per(std[0], 2), '年化下行标准差*': to_per(std_star[0], 2), "最大回撤": MaxDrawdown, "回补周期(天)": restore_duration, 
             '夏普比率': round(sharpe.iloc[0], 2), '索提诺': round(sortino.iloc[0], 2), '索提诺*': round(sortino_star.iloc[0], 2), '卡玛比率':round(calmar.iloc[0], 2), 
             '月胜率': str(round(monthly))+"%",'周胜率': str(round(weekly))+"%"}, ignore_index=True)


        full_return_yearly = empyrical.aggregate_returns(df1[i], 'yearly')/ df1.shape[0] * 52
        full_return = empyrical.aggregate_returns(df1[i], 'yearly')
        full_anl_std = df1[i].std() * (52 ** 0.5)
        full_std = s_std(df1)
        full_std_star = s_std_star(df1)
        full_MaxDrawdown, start_date, end_date, restore_end, full_restore_duration = max_drawdown(df2)
        
        full_sharpe = full_sharpe_ratio(df1)
        full_sortino = full_sortino_ratio(df1)
        full_sortino_star = full_sortino_ratio(df1)

        
        full_max_dd = empyrical.max_drawdown(df1[str(i)])
        full_calmar = -1 * (full_return_yearly / full_max_dd)


        full_monthly = monthly_rate(df1)
        full_weekly = weekly_rate(df1)

        if full_MaxDrawdown!= "NA":
                full_MaxDrawdown = str(round(full_MaxDrawdown*100,2))+"%"


        result = result.append({'产品': i, '年份':"全周期", '收益率': to_per(full_return.iloc[0], 2), '年化收益率': to_per(full_return_yearly.iloc[0], 2),  '年化波动率': to_per(full_anl_std, 2), 
        '年化下行标准差': to_per(full_std[0], 2), '年化下行标准差*': to_per(full_std_star[0], 2),  "最大回撤": full_MaxDrawdown, "回补周期(天)": full_restore_duration, 
        '夏普比率': round(sharpe.iloc[0], 2), '索提诺': round(full_sortino.iloc[0], 2), '索提诺*': round(full_sortino_star.iloc[0], 2), '卡玛比率':round(full_calmar.iloc[0], 2), 
        '月胜率': str(round(monthly))+"%", '周胜率': str(round(weekly))+"%"}, ignore_index=True)
    

    yr_sort = CategoricalDtype([2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, "全周期"], ordered=True) 
    index_sort = CategoricalDtype(['收益率', '年化收益率', '年化波动率', '年化下行标准差', '年化下行标准差*', "最大回撤", "回补周期(天)", 
    '夏普比率', '索提诺', '索提诺*', '卡玛比率', '月胜率', '周胜率'], ordered=True)




    result_levels = result.set_index(['年份', '产品'])
    result_levels = result_levels.unstack(0)
    result_levels= result_levels.T.swaplevel(0, 1).reset_index()

    result_levels["年份"] = result_levels["年份"].astype(yr_sort)
    result_levels["level_1"] = result_levels["level_1"].astype(index_sort)

    result_levels.sort_values(by=["年份", "level_1"], inplace=True)
    result_levels = result_levels.set_index(['年份', "level_1"]).T

 
           

    result_levels.to_excel(write, sheet_name="指标", index=True) 

    monthly_return.columns = [str(datetime.datetime.strptime(str(i[0])+"-W"+str(i[1])+"-5", "%Y-W%W-%w").date()) for i in monthly_return.columns]
    monthly_return.sort_index(axis=1, ascending=False, inplace=True)

    weekly_return.columns = [str(datetime.datetime.strptime(str(i[0])+"-W"+str(i[1])+"-5", "%Y-W%W-%w").date()) for i in weekly_return.columns]
    weekly_return.sort_index(axis=1, ascending=False, inplace=True)

    weekly_return.to_excel(write, sheet_name="周收益率", index=True)
    monthly_return.to_excel(write, sheet_name="月收益率", index=True)


    
    write.save()

    


In [ ]:
# to cb indexes


from pandas.api.types import CategoricalDtype
import datetime


for k in ["Sheet2"]:

    write = pd.ExcelWriter('../data/净值月报/可转债.xlsx')

    df = pd.read_excel('../data/可转债净值.xlsx',sheet_name = k)
    name = np.delete(df.columns.values,0)
    weekly_return = pd.DataFrame()
    monthly_return = pd.DataFrame()
    result = pd.DataFrame()

    for i in name:
        df1 = df.loc[:, ['日期',i]]
        df1 = df1.dropna()
        df1 = df1.reset_index(drop=True)
        df1 = df1.set_index('日期')
        df2 = df1.copy()

        df1['exe_return'] = df1[i]
        df1['prev_return'] = df1[i].shift(1)
        df1[i] = (df1['exe_return'] - df1['prev_return']) / df1['prev_return']
        df1 = df1.drop(['prev_return', 'exe_return'], axis=1)
        df1 = df1.dropna()

        new_weekly_return = empyrical.aggregate_returns(df1[i], 'weekly')
        weekly_return = weekly_return.append(new_weekly_return)

        new_monthly_return = empyrical.aggregate_returns(df1[i], 'monthly')
        monthly_return = monthly_return.append(new_monthly_return)

        for j in (df1.index.year).unique():
            aggregate_return_yearly = empyrical.aggregate_returns(df1.loc[str(j)][str(i)], 'yearly')/ df1.loc[str(j)].shape[0] * 52
            aggregate_return = empyrical.aggregate_returns(df1.loc[str(j)][str(i)], 'yearly')

            # 年化波动率
            anl_std = df1[str(i)][str(j)].std() * (52 ** 0.5)
            df_year = df1.loc[str(j)]


            std = s_std(df_year)
            std_star = s_std_star(df_year)


            monthly = monthly_rate(df_year)
            weekly = weekly_rate(df_year)

            sharpe = sharpe_ratio(df_year)
            sortino = sortino_ratio(df_year)
            sortino_star = sortino_ratio(df_year)
        
            max_dd = empyrical.max_drawdown(df1[str(i)][str(j)])
            calmar = -1 * (aggregate_return_yearly / max_dd)
            
            df_mdd = df2[df2.index.year==j]
            MaxDrawdown, start_date, end_date, restore_end, restore_duration = max_drawdown(df_mdd)

            if MaxDrawdown!= "NA":
                MaxDrawdown = str(round(MaxDrawdown*100,2))+"%"
            
            result = result.append({'产品': i, '年份':j, '收益率': to_per(aggregate_return.iloc[0], 2), '年化收益率': to_per(aggregate_return_yearly.iloc[0], 2),  '年化波动率': to_per(anl_std, 2), 
             '年化下行标准差': to_per(std[0], 2), '年化下行标准差*': to_per(std_star[0], 2), "最大回撤": MaxDrawdown, "回补周期(天)": restore_duration, 
             '夏普比率': round(sharpe.iloc[0], 2), '索提诺': round(sortino.iloc[0], 2), '索提诺*': round(sortino_star.iloc[0], 2), '卡玛比率':round(calmar.iloc[0], 2), 
             '月胜率': str(round(monthly))+"%",'周胜率': str(round(weekly))+"%"}, ignore_index=True)


        full_return_yearly = empyrical.aggregate_returns(df1[i], 'yearly')/ df1.shape[0] * 52
        full_return = empyrical.aggregate_returns(df1[i], 'yearly')
        full_anl_std = df1[i].std() * (52 ** 0.5)
        full_std = s_std(df1)
        full_std_star = s_std_star(df1)
        full_MaxDrawdown, start_date, end_date, restore_end, full_restore_duration = max_drawdown(df2)
        
        full_sharpe = full_sharpe_ratio(df1)
        full_sortino = full_sortino_ratio(df1)
        full_sortino_star = full_sortino_ratio(df1)

        
        full_max_dd = empyrical.max_drawdown(df1[str(i)])
        full_calmar = -1 * (full_return_yearly / full_max_dd)


        full_monthly = monthly_rate(df1)
        full_weekly = weekly_rate(df1)

        if full_MaxDrawdown!= "NA":
                full_MaxDrawdown = str(round(full_MaxDrawdown*100,2))+"%"


        result = result.append({'产品': i, '年份':"全周期", '收益率': to_per(full_return.iloc[0], 2), '年化收益率': to_per(full_return_yearly.iloc[0], 2),  '年化波动率': to_per(full_anl_std, 2), 
        '年化下行标准差': to_per(full_std[0], 2), '年化下行标准差*': to_per(full_std_star[0], 2),  "最大回撤": str(max_dd*100)+"%", "最大回撤": full_MaxDrawdown, "回补周期(天)": full_restore_duration, 
        '夏普比率': round(sharpe.iloc[0], 2), '索提诺': round(full_sortino.iloc[0], 2), '索提诺*': round(full_sortino_star.iloc[0], 2), '卡玛比率':round(full_calmar.iloc[0], 2), 
        '月胜率': str(round(monthly))+"%",'周胜率': str(round(weekly))+"%"}, ignore_index=True)
    
    yr_sort = CategoricalDtype([2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, "全周期"], ordered=True) 
    index_sort = CategoricalDtype(['收益率', '年化收益率', '年化波动率', '年化下行标准差', '年化下行标准差*', "最大回撤", "回补周期(天)", 
    '夏普比率', '索提诺', '索提诺*', '卡玛比率', '月胜率', '周胜率'], ordered=True)


    result_levels = result.set_index(['年份', '产品'])
    result_levels = result_levels.unstack(0)
    result_levels= result_levels.T.swaplevel(0, 1).reset_index()

    result_levels["年份"] = result_levels["年份"].astype(yr_sort)
    result_levels["level_1"] = result_levels["level_1"].astype(index_sort)

    result_levels.sort_values(by=["年份", "level_1"], inplace=True)
    result_levels = result_levels.set_index(['年份', "level_1"]).T

 
           

    result_levels.to_excel(write, sheet_name="指标", index=True) 

    monthly_return.columns = [str(datetime.datetime.strptime(str(i[0])+"-W"+str(i[1])+"-5", "%Y-W%W-%w").date()) for i in monthly_return.columns]
    monthly_return.sort_index(axis=1, ascending=False, inplace=True)

    weekly_return.columns = [str(datetime.datetime.strptime(str(i[0])+"-W"+str(i[1])+"-5", "%Y-W%W-%w").date()) for i in weekly_return.columns]
    weekly_return.sort_index(axis=1, ascending=False, inplace=True)

    weekly_return.to_excel(write, sheet_name="周收益率", index=True)
    monthly_return.to_excel(write, sheet_name="月收益率", index=True)


    write.save()
   

In [ ]:
# 锐天核心8 #改路径

import datetime

for k in ["Sheet1"]:

    write = pd.ExcelWriter('../data/净值月报/锐天.xlsx')

    df = pd.read_excel('../data/核心8.xlsx',sheet_name = k)
    name = np.delete(df.columns.values,0)
    weekly_return = pd.DataFrame()
    monthly_return = pd.DataFrame()
    result = pd.DataFrame()

    for i in name:
        df1 = df.loc[:, ['日期',i]]
        df1 = df1.dropna()
        df1 = df1.reset_index(drop=True)
        df1 = df1.set_index('日期')
        df2 = df1.copy()

        df1['exe_return'] = df1[i]
        df1['prev_return'] = df1[i].shift(1)
        df1[i] = (df1['exe_return'] - df1['prev_return']) / df1['prev_return']
        df1 = df1.drop(['prev_return', 'exe_return'], axis=1)
        df1 = df1.dropna()

        new_weekly_return = empyrical.aggregate_returns(df1[i], 'weekly')
        #print(new_weekly_return)
        weekly_return = weekly_return.append(new_weekly_return)

        new_monthly_return = empyrical.aggregate_returns(df1[i], 'monthly')
        monthly_return = monthly_return.append(new_monthly_return)

        for j in (df1.index.year).unique():
            aggregate_return_yearly = empyrical.aggregate_returns(df1.loc[str(j)][str(i)], 'yearly')/ df1.loc[str(j)].shape[0] * 52
            aggregate_return = empyrical.aggregate_returns(df1.loc[str(j)][str(i)], 'yearly')

            # 年化波动率
            anl_std = df1[str(i)][str(j)].std() * (52 ** 0.5)
            df_year = df1.loc[str(j)]


            std = s_std(df_year)
            std_star = s_std_star(df_year)


            monthly = monthly_rate(df_year)
            weekly = weekly_rate(df_year)

            sharpe = sharpe_ratio(df_year)
            sortino = sortino_ratio(df_year)
            sortino_star = sortino_ratio(df_year)
        
            max_dd = empyrical.max_drawdown(df1[str(i)][str(j)])
            calmar = -1 * (aggregate_return_yearly / max_dd)
            
            df_mdd = df2[df2.index.year==j]
            MaxDrawdown, start_date, end_date, restore_end, restore_duration = max_drawdown(df_mdd)

            if MaxDrawdown!= "NA":
                MaxDrawdown = str(round(MaxDrawdown*100,2))+"%"
            
            result = result.append({'产品': i, '年份':j, '收益率': to_per(aggregate_return.iloc[0], 2), '年化收益率': to_per(aggregate_return_yearly.iloc[0], 2),  '年化波动率': to_per(anl_std, 2), 
             '年化下行标准差': to_per(std[0], 2), '年化下行标准差*': to_per(std_star[0], 2), "最大回撤": MaxDrawdown, "回补周期(天)": restore_duration, 
             '夏普比率': round(sharpe.iloc[0], 2), '索提诺': round(sortino.iloc[0], 2), '索提诺*': round(sortino_star.iloc[0], 2), '卡玛比率':round(calmar.iloc[0], 2), 
             '月胜率': str(round(monthly))+"%",'周胜率': str(round(weekly))+"%"}, ignore_index=True)


        full_return_yearly = empyrical.aggregate_returns(df1[i], 'yearly')/ df1.shape[0] * 52
        full_return = empyrical.aggregate_returns(df1[i], 'yearly')
        full_anl_std = df1[i].std() * (52 ** 0.5)
        full_std = s_std(df1)
        full_std_star = s_std_star(df1)
        full_MaxDrawdown, start_date, end_date, restore_end, full_restore_duration = max_drawdown(df2)
        
        full_sharpe = full_sharpe_ratio(df1)
        full_sortino = full_sortino_ratio(df1)
        full_sortino_star = full_sortino_ratio(df1)

        
        full_max_dd = empyrical.max_drawdown(df1[str(i)])
        full_calmar = -1 * (full_return_yearly / full_max_dd)


        full_monthly = monthly_rate(df1)
        full_weekly = weekly_rate(df1)

        if full_MaxDrawdown!= "NA":
                full_MaxDrawdown = str(round(full_MaxDrawdown*100,2))+"%"


        result = result.append({'产品': i, '年份':"全周期", '收益率': to_per(full_return.iloc[0], 2), '年化收益率': to_per(full_return_yearly.iloc[0], 2),  '年化波动率': to_per(full_anl_std, 2), 
        '年化下行标准差': to_per(full_std[0], 2), '年化下行标准差*': to_per(full_std_star[0], 2),  "最大回撤": str(max_dd*100)+"%", "最大回撤": full_MaxDrawdown, "回补周期(天)": full_restore_duration, 
        '夏普比率': round(sharpe.iloc[0], 2), '索提诺': round(full_sortino.iloc[0], 2), '索提诺*': round(full_sortino_star.iloc[0], 2), '卡玛比率':round(full_calmar.iloc[0], 2), 
        '月胜率': str(round(monthly))+"%",'周胜率': str(round(weekly))+"%"}, ignore_index=True)
    
    yr_sort = CategoricalDtype([2022, 2021, 2020, 2019, 2018, 2017, 2016, 2015, "全周期"], ordered=True) 
    index_sort = CategoricalDtype(['收益率', '年化收益率', '年化波动率', '年化下行标准差', '年化下行标准差*', "最大回撤", "回补周期(天)", 
    '夏普比率', '索提诺', '索提诺*', '卡玛比率', '月胜率', '周胜率'], ordered=True)


    result_levels = result.set_index(['年份', '产品'])
    result_levels = result_levels.unstack(0)
    result_levels= result_levels.T.swaplevel(0, 1).reset_index()

    result_levels["年份"] = result_levels["年份"].astype(yr_sort)
    result_levels["level_1"] = result_levels["level_1"].astype(index_sort)

    result_levels.sort_values(by=["年份", "level_1"], inplace=True)
    result_levels = result_levels.set_index(['年份', "level_1"])
    result_levels.T.to_excel(write, sheet_name="指标", index=True) 

  
    monthly_return.columns = [str(datetime.datetime.strptime(str(i[0])+"-W"+str(i[1])+"-5", "%Y-W%W-%w").date()) for i in monthly_return.columns]
    monthly_return.sort_index(axis=1, ascending=False, inplace=True)

    weekly_return.columns = [str(datetime.datetime.strptime(str(i[0])+"-W"+str(i[1])+"-5", "%Y-W%W-%w").date()) for i in weekly_return.columns]
    weekly_return.sort_index(axis=1, ascending=False, inplace=True)


    weekly_return.to_excel(write, sheet_name="周收益率", index=True)
    monthly_return.to_excel(write, sheet_name="月收益率", index=True)

    write.save()
    
    
    
    